# RAG Evaluation

Retrieval Augmented Generation (RAG) is a technique that enhances Large Language Models (LLMs) by providing them with relevant external knowledge. It has become one of the most widely used approaches for building LLM applications.

This tutorial will show you how to evaluate your RAG applications using LangSmith. You'll learn:

1. How to create test datasets
2. How to run your RAG application on those datasets
3. How to measure your application's performance using different evaluation metrics

## Overview
A typical RAG evaluation workflow consists of three main steps:

1. Creating a dataset with questions and their expected answers
2. Running your RAG application on those questions
3. Using evaluators to measure how well your application performed, looking at factors like:
 - Answer relevance
 - Answer accuracy
 - Retrieval quality

## Setup and Import Libraries

In [1]:
import os
import openai
from typing_extensions import Annotated,TypedDict
from langchain.chat_models import init_chat_model
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langsmith import Client, wrappers, traceable


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from dotenv import load_dotenv

In [3]:
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"

In [4]:
llm = init_chat_model("openai:gpt-4o-mini")
embedding = OpenAIEmbeddings()

## Create Retriver

In [5]:
def create_retriever(chunk_size=250, chunk_overlap=0, k=6):
    # List of URLs to load documents from
    urls = [
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]

    # Load documents from the URLs
    documents = [WebBaseLoader(url).load() for url in urls]
    document_list = [item for sublist in documents for item in sublist]

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    # Split the documents into chunks
    chunks = text_splitter.split_documents(document_list)

    # Add the document chunks to the "vector store" using OpenAIEmbeddings
    vectorstore = InMemoryVectorStore.from_documents(
        documents=chunks,
        embedding=embedding,
    )

    retriever = vectorstore.as_retriever(k=k)
    return retriever

In [6]:
retriever = create_retriever()

retriever.invoke("what is agents")

[Document(id='dfa71433-50e8-4d90-b3f8-c5f21d2c1fd9', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, 

## Define Generation Pipeline

In [7]:
@traceable()
def rag_bot(question:str) -> dict:
    retrieved_documents = retriever.invoke(question)
    result = " ".join(document.page_content for document in retrieved_documents)

    instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.       
    Use the following source documents to answer the user's questions.       
    If you don't know the answer, just say that you don't know.       
    Use three sentences maximum and keep the answer concise.
    
    Documents:
    {result}
    """

    response = llm.invoke([
        {"role": "system", "content": instructions},
        {"role": "user", "content": question},
    ])

    return {"answer":response.content,"documents":retrieved_documents}

In [8]:
rag_bot("What is agents")

{'answer': 'Agents are autonomous entities that operate based on predefined behaviors, making decisions and interacting with their environment or other agents. They often employ mechanisms like memory, planning, and reflection to enhance their effectiveness. In the context of generative agents, these are virtual characters powered by large language models that simulate human-like behavior in interactive environments.',
 'documents': [Document(id='dfa71433-50e8-4d90-b3f8-c5f21d2c1fd9', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System O

## Create Test Dataset

In [9]:
langsmith_client = Client()

In [10]:
examples = [
    {
        "inputs": {"question": "How does the ReAct agent use self-reflection? "},
        "outputs": {"answer": "ReAct integrates reasoning and acting, performing actions - such tools like Wikipedia search API - and then observing / reasoning about the tool outputs."},
    },
    {
        "inputs": {"question": "What are the types of biases that can arise with few-shot prompting?"},
        "outputs": {"answer": "The biases that can arise with few-shot prompting include (1) Majority label bias, (2) Recency bias, and (3) Common token bias."},
    },
    {
        "inputs": {"question": "What are five types of adversarial attacks?"},
        "outputs": {"answer": "Five types of adversarial attacks are (1) Token manipulation, (2) Gradient based attack, (3) Jailbreak prompting, (4) Human red-teaming, (5) Model red-teaming."},
    }
]

In [11]:
dataset_name="RAG Test Evaluation"

dataset = langsmith_client.create_dataset(dataset_name=dataset_name)

langsmith_client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)

{'example_ids': ['64576064-a2bd-45e1-823b-1bd05f3d6d86',
  '74d50293-160a-4acf-ae64-49da0f752031',
  '4f5e5792-a385-4c4f-8d9e-4d623718fddd'],
 'count': 3}

## Evaluation Metrics

### 1. Correctness: Response vs reference answer
- Goal: Measure "how similar/correct is the RAG chain answer, relative to a ground-truth answer"
- Mode: Requires a ground truth (reference) answer supplied through a dataset
- Evaluator: Use LLM-as-judge to assess answer correctness.

In [12]:
# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

In [13]:
correctness_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. 
(2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

In [14]:
correctness_llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0
).with_structured_output(
    schema=CorrectnessGrade, method="json_schema", strict=True
)

In [15]:
def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""
    QUESTION: {inputs['question']}
    GROUND TRUTH ANSWER: {reference_outputs['answer']}
    STUDENT ANSWER: {outputs['answer']}"""

    # Run evaluator
    grade = correctness_llm.invoke([
        {"role": "system", "content": correctness_instructions}, 
        {"role": "user", "content": answers}
    ])
    
    return grade["correct"]

### 2. Relevance: Response vs input

- The flow is similar to above, but we simply look at the inputs and outputs without needing the reference_outputs. 
- Without a reference answer we can't grade accuracy, but can still grade relevance—as in, did the model address the user's question or not.

In [16]:
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

In [17]:
relevance_instructions="""You are a teacher grading a quiz. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset.
"""

In [18]:
relevance_llm = ChatOpenAI(
    model="gpt-4o", temperature=0).with_structured_output(
        schema=RelevanceGrade, method="json_schema", strict=True
    )

In [19]:
def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"QUESTION: {inputs['question']}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

### 3. Groundedness: Response vs retrieved docs

- Another useful way to evaluate responses without needing reference answers is to check if the response is justified by (or "grounded in") the retrieved documents.

In [20]:
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]


In [21]:
grounded_instructions = """You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 
(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

In [22]:
grounded_llm = ChatOpenAI(
    model="gpt-4o", temperature=0).with_structured_output(
        schema=GroundedGrade, method="json_schema", strict=True
    )

In [23]:
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    result = "\n\n".join(document.page_content for document in outputs["documents"])
    answer = f"FACTS: {result}\nSTUDENT ANSWER: {outputs['answer']}"

    grade = grounded_llm.invoke(
        [
            {"role": "system", "content": grounded_instructions}, 
            {"role": "user", "content": answer}
        ]
    )
    return grade["grounded"]

### 4. Retrieval Relevance: Retrieved docs vs input

In [25]:
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

In [26]:
retrieval_relevance_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a set of FACTS provided by the student. 

Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

In [27]:
retrieval_relevance_llm = ChatOpenAI(
    model="gpt-4o", temperature=0).with_structured_output(
        schema=RetrievalRelevanceGrade, method="json_schema", strict=True
    )

In [28]:
def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    result = "\n\n".join(document.page_content for document in outputs["documents"])
    answer = f"FACTS: {result}\nQUESTION: {inputs['question']}"

    # Run evaluator
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

## Run the Evaluation

In [29]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

In [32]:
experiment_results = langsmith_client.evaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance",
    metadata={"version": "LCEL context, gpt-4-0125-preview"},
)

View the evaluation results for experiment: 'rag-doc-relevance-e80fe443' at:
https://smith.langchain.com/o/e70f5ce4-4aca-4a02-b6ba-8f13fc21b177/datasets/f85cef72-7e4f-4250-92e2-da5684f69361/compare?selectedSessions=f659c58e-b328-4d84-bc33-be579dbcf9b7




3it [00:33, 11.08s/it]


In [33]:
experiment_results.to_pandas()

,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,feedback.relevance,feedback.retrieval_relevance,execution_time,example_id,id
0,What are five types of adversarial attacks?,The five types of adversarial attacks are:\n\n...,[page_content='Black-box attacks assume that a...,None,Five types of adversarial attacks are (1) Toke...,True,True,True,True,3.636480,4f5e5792-a385-4c4f-8d9e-4d623718fddd,ca66f8ff-6a00-4c94-9665-1775a7e2a8e3
1,How does the ReAct agent use self-reflection?,The ReAct agent incorporates self-reflection b...,[page_content='Self-reflection is a vital aspe...,None,"ReAct integrates reasoning and acting, perform...",False,False,True,False,2.409996,64576064-a2bd-45e1-823b-1bd05f3d6d86,746fff8e-a11d-4b84-8cad-be8dcf51320b
2,What are the types of biases that can arise wi...,The types of biases that can arise with few-sh...,[page_content='Zero-shot and few-shot learning...,None,The biases that can arise with few-shot prompt...,True,True,True,True,1.969844,74d50293-160a-4acf-ae64-49da0f752031,cbc4000b-b8fd-49a8-853b-f99d1aace6f0
